# スクレイピング(2025/07/01) 

### ■ 使い方
1. 下の「設定」セルで、`TARGET_URL`, `SAVE_DIR`, `SAVE_FORMAT` の3つの変数を、ご自身の環境に合わせて編集してください。
2. **「設定」セルを実行**します。（セルを選択して `Shift + Enter`）
3. **「スクレイピング実行」セルを実行**します。（セルを選択して `Shift + Enter`）

In [ ]:
import os

TARGET_URL = "https://www.ap-siken.com/apkakomon.php"

SAVE_DIR = "D:\\steganography\\manga"  # 例: "C:\\Users\\YourUser\\Downloads\\scraped_images"

SAVE_FORMAT = "jpg"

if not os.path.isdir(SAVE_DIR):
    print(f"エラー: 保存先ディレクトリ '{SAVE_DIR}' が存在しません。")
    print("上記の 'SAVE_DIR' を有効なパスに修正するか、ディレクトリを事前に作成してください。")
    # このエラーが出た場合は、以降のセル実行はするな
else:
    print("設定が読み込まれました。次のセルを実行してスクレイピングを開始してください。")
    print(f"  対象URL: {TARGET_URL}")
    print(f"  保存先: {SAVE_DIR}")
    print(f"  保存形式: .{SAVE_FORMAT}")

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

if not os.path.isdir(SAVE_DIR):
    print(f"エラー: 保存先ディレクトリ '{SAVE_DIR}' が見つかりません。上の設定セルを修正・実行してください。")
else:
    print(f"\n'{TARGET_URL}' からHTMLを取得しています...")
    try:
        response = requests.get(TARGET_URL, timeout=15)
        response.raise_for_status()  # ステータスコードが200番台以外の場合は例外を発生させるらしい
        
    except requests.exceptions.RequestException as e:
        print(f"エラー: Webページの取得に失敗しました。 {e}")

    else:
        soup = BeautifulSoup(response.content, "html.parser")
        img_tags = soup.find_all("img")
        
        if not img_tags:
            print("ページ内に画像が見つかりませんでした。")

        else:
            print(f"{len(img_tags)} 個の画像が見つかりました。ダウンロードを開始します。")

            saved_count = 0
            for i, img in enumerate(img_tags, start=1):
                img_url = img.get("src")
                if not img_url:
                    print(f"警告: {i}番目のimgタグにはsrc属性がありませんでした。スキップします。")
                    continue

                img_url_absolute = urljoin(TARGET_URL, img_url)

                try:
                    print(f"({i}/{len(img_tags)}) ダウンロード中: {img_url_absolute}")
                    img_data = requests.get(img_url_absolute, timeout=10).content
                    
                    filename_from_url = os.path.basename(urljoin(img_url_absolute, '.'))
                    filename_from_url = filename_from_url.split('?')[0]
                    base_name, _ = os.path.splitext(filename_from_url)
                    
                    if base_name:
                        filename = f"{i:03d}_{base_name}.{SAVE_FORMAT}"
                    else:
                        filename = f"image_{i:03d}.{SAVE_FORMAT}"

                    save_path = os.path.join(SAVE_DIR, filename)

                    with open(save_path, "wb") as f:
                        f.write(img_data)
                    saved_count += 1
                    
                    time.sleep(0.5)

                except requests.exceptions.RequestException as e:
                    print(f"エラー: {img_url_absolute} のダウンロードに失敗しました: {e}")

                except Exception as e:
                    print(f"エラー: {img_url_absolute} の保存中に予期せぬエラーが発生しました: {e}")

            print(f"\n完了しました。{saved_count}個の画像を'{SAVE_DIR}'に保存しました。")